## as the last model has 0.934 train accuracy and 0.734 test accuracy it is clearly overfitting so I'll try regulazation 

In [ ]:
from dnn_structure import *

In [ ]:
from dnn_structure import *

import numpy as np
import sys
sys.path.append("../src")
from Load_data.xml_utils import parse_cvat_xml_all_labels, label_Y_binary
from Load_data.data_loader import load_and_resize_images, build_label_array


In [ ]:

TRAIN_XML = "../EIDSeg_Dataset/data/train/train.xml"
TEST_XML = "../EIDSeg_Dataset/data/test/test.xml"
TRAIN_IMAGES = "../EIDSeg_Dataset/data/train/images/default"
TEST_IMAGES = "../EIDSeg_Dataset/data/test/images/default"

IMAGE_SIZE = (64, 64)
NUM_ITER = 2000



labels_train_raw = parse_cvat_xml_all_labels(TRAIN_XML)
labels_test_raw = parse_cvat_xml_all_labels(TEST_XML)

Y_train_map = label_Y_binary(labels_train_raw)
Y_test_map = label_Y_binary(labels_test_raw)

X_train_org, ordered_train = load_and_resize_images(TRAIN_IMAGES, size=IMAGE_SIZE)
X_test_org, ordered_test = load_and_resize_images(TEST_IMAGES, size=IMAGE_SIZE)

Y_train_org = build_label_array(ordered_train, Y_train_map)
Y_test_org = build_label_array(ordered_test, Y_test_map)

train_x = X_train_org.reshape(X_train_org.shape[0], -1).T
test_x = X_test_org.reshape(X_test_org.shape[0], -1).T

print(train_x.shape, Y_train_org.shape)
print(test_x.shape, Y_test_org.shape)

## L2 Regularization

In [ ]:
def compute_cost_with_regularization(AL, Y, parameters, lambd):
    """
    Implement the cost function with L2 regularization.

    Arguments:
    AL -- probability vector from forward propagation, shape (1, m)
    Y -- true labels vector, shape (1, m)
    parameters -- python dictionary containing W1...WL
    lambd -- regularization parameter

    Returns:
    cost -- regularized cost
    """
    m = Y.shape[1]
    
    # Cross-entropy cost
    cross_entropy_cost = compute_cost(AL, Y)
        
    # L2 regularization cost
    L2_regularization_cost = 0
    L = len(parameters) // 2  # number of layers
    
    for l in range(1, L + 1):
        L2_regularization_cost += np.sum(np.square(parameters["W" + str(l)]))
    
    L2_regularization_cost *= (lambd / (2 * m))
    
    cost = cross_entropy_cost + L2_regularization_cost
    
    return cost


In [ ]:
def L_model_backward_with_regularization(AL, Y, caches, lambd):
    """
    Implement backward propagation with L2 regularization for [LINEAR->RELU]*(L-1)->LINEAR->SIGMOID
    
    Arguments:
    AL -- probability vector, output of forward propagation (1, m)
    Y -- true labels vector (1, m)
    caches -- list of caches from L_model_forward
    lambd -- regularization parameter
    
    Returns:
    grads -- dictionary with gradients
    """
    
    grads = {}
    L = len(caches)           # number of layers
    m = AL.shape[1]
    Y = Y.reshape(AL.shape)
    
    # ---------- OUTPUT LAYER (SIGMOID) ----------
    current_cache = caches[L - 1]
    
    dAL = - (np.divide(Y, AL) - np.divide(1 - Y, 1 - AL))
    
    dA_prev, dW, db = linear_activation_backward(
        dAL, current_cache, activation="sigmoid"
    )
    
    grads["dW" + str(L)] = dW + (lambd / m) * current_cache[0][1]
    grads["db" + str(L)] = db
    grads["dA" + str(L - 1)] = dA_prev
    
    # ---------- HIDDEN LAYERS (RELU) ----------
    for l in reversed(range(L - 1)):
        current_cache = caches[l]
        
        dA_prev, dW, db = linear_activation_backward(
            grads["dA" + str(l + 1)], current_cache, activation="relu"
        )
        
        grads["dW" + str(l + 1)] = dW + (lambd / m) * current_cache[0][1]
        grads["db" + str(l + 1)] = db
        grads["dA" + str(l)] = dA_prev
    
    return grads


In [ ]:

def L_layer_model(X, Y, layers_dims, learning_rate = 0.0075, num_iterations = 3000, print_cost=False):
    """
    Implements a L-layer neural network: [LINEAR->RELU]*(L-1)->LINEAR->SIGMOID.
    
    Arguments:
    X -- input data, of shape (n_x, number of examples)
    Y -- true "label" vector (containing 1 if cat, 0 if non-cat), of shape (1, number of examples)
    layers_dims -- list containing the input size and each layer size, of length (number of layers + 1).
    learning_rate -- learning rate of the gradient descent update rule
    num_iterations -- number of iterations of the optimization loop
    print_cost -- if True, it prints the cost every 100 steps
    
    Returns:
    parameters -- parameters learnt by the model. They can then be used to predict.
    """

    np.random.seed(1)
    costs = []                         # keep track of cost
    
    # Parameters initialization.

    parameters = initialize_parameters_deep(layers_dims)
        
    # Loop (gradient descent)
    for i in range(0, num_iterations):
        # Forward propagation: [LINEAR -> RELU]*(L-1) -> LINEAR -> SIGMOID.
        AL, caches = L_model_forward(X, parameters)
                
        # Compute cost.
        cost = compute_cost_with_regularization(AL, Y, parameters, lambd=0.7)
            
        # Backward propagation.
        grads = L_model_backward_with_regularization(AL, Y, caches, lambd=0.7)
        
        # Update parameters.

        parameters = update_parameters(parameters, grads, learning_rate)
                        
        # Print the cost every 100 iterations and for the last iteration
        if print_cost and (i % 100 == 0 or i == num_iterations - 1):
            print("Cost after iteration {}: {}".format(i, np.squeeze(cost)))
        if i % 100 == 0:
            costs.append(cost)
    
    return parameters, costs

In [ ]:
layers_dims = [12288, 20, 7, 5, 1] 

In [ ]:
parameters, costs = L_layer_model(train_x, Y_train_org, layers_dims,learning_rate = 0.008, num_iterations = 3500, print_cost = True)